In [2]:
import logging
from sklearn.linear_model import LinearRegression
import wandb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import os,sys,yaml
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

if module_path not in sys.path:
    sys.path.append(module_path)
# Now you can import your module
from src.DataOps.feature_engg import feature_engg_class
from src.ModelOps.model_fit import ModelFit
from src.DataOps.feature_engg import feature_engg_class

In [18]:
os.environ["WANDB_API_KEY"] = "22787bdec6329d031c43de72471e610b908a8815"
with open('conf/mlops.yaml', 'r') as file:
            config = yaml.safe_load(file)
model_name = config['model_name']
project_name=config['project_name']
model_type=config['model_type']
run = wandb.init(project="ml-ops-template",job_type="inference")

In [19]:
model_name

'diabetes_model'

In [20]:
artifact = run.use_artifact(f'sakshamgulati123/model-registry/{model_name}:v0', type='model')
# downloaded_model_path = run.use_model(name=model_name)
print(artifact)

artifact_dir = artifact.download()
print(artifact_dir)

<Artifact QXJ0aWZhY3Q6ODA5ODc1MDAy>


wandb:   1 of 1 files downloaded.  


/Users/sakshamgulati/MLOps_Template/artifacts/run-c6p9lia4-model.pickle:v0


In [26]:



diabetes = feature_engg_class()
data, target = diabetes.load_data()
fin_df = diabetes.standard_scaling(data)
X_train, X_test, y_train, y_test = diabetes.split(
    fin_df, target
)
model = RandomForestRegressor().fit(X_train, y_train)


INFO:root:Data loaded from sklearn datasets
INFO:root:Data imputed for missing values
INFO:root:Data Scaled
INFO:root:Data split into testing and train


In [18]:
from pathlib import Path
filename = Path("artifacts/model1/model.pickle")
# Ensure the directory exists
filename.parent.mkdir(parents=True, exist_ok=True)
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [19]:
registered_model_name = "MNIST-dev"
run.link_model(path=filename, registered_model_name=registered_model_name)
run.finish()

In [20]:
job_type = "evaluate_model"

# Initialize a run
run = wandb.init(project="ml-ops-template",job_type="inference")

artifact = run.use_artifact('sakshamgulati123/ml-ops-template/run-huppgypw-model.pickle:v0', type='model')
artifact_dir = artifact.download()
print(artifact_dir)

wandb:   1 of 1 files downloaded.  


/Users/sakshamgulati/MLOps_Template/artifacts/run-huppgypw-model.pickle:v0


In [21]:
file_path = os.path.join(artifact_dir, 'model.pickle')  # specify the correct file path
with open(file_path, 'rb') as f:
    model = pickle.load(f)
logging.info("Model loaded from the registry")

INFO:root:Model loaded from the registry


In [30]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# logging MSE and R2 score
logging.info(f"MSE:{mse}")
logging.info(f"R2:{r2}")
wandb.log({"Mean squared error": mse, "Test R-squared": r2})

INFO:root:MSE:3079.8153815068495
INFO:root:R2:0.46486945233875165
